# STEP 1 - import and setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm


# STEP 2 - 

In [2]:
def get_resnet18_cifar():
    model = models.resnet18(weights=None)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    model.fc = nn.Linear(512, 10)
    return model

def fgsm_attack(model, images, labels, epsilon):
    images.requires_grad = True
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)

    model.zero_grad()
    loss.backward()

    perturbation = epsilon * images.grad.sign()
    adv_images = images + perturbation
    adv_images = torch.clamp(adv_images, 0, 1)
    return adv_images.detach()


In [3]:
device = torch.device("cpu")  # locale su Mac

model = get_resnet18_cifar()
model.load_state_dict(torch.load("resnet18_adversarial_trained.pth", map_location=device))
model.to(device)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [4]:
transform = transforms.Compose([transforms.ToTensor()])
testset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False)


In [5]:
epsilon = 0.1
correct_clean = 0
correct_adv = 0
total = 0

for images, labels in tqdm(testloader):
    images, labels = images.to(device), labels.to(device)
    total += labels.size(0)

    # clean accuracy
    output_clean = model(images)
    pred_clean = output_clean.argmax(dim=1)
    correct_clean += (pred_clean == labels).sum().item()

    # adversarial accuracy
    adv_images = fgsm_attack(model, images.clone(), labels, epsilon)
    output_adv = model(adv_images)
    pred_adv = output_adv.argmax(dim=1)
    correct_adv += (pred_adv == labels).sum().item()

print(f"Clean Accuracy:      {100 * correct_clean / total:.2f}%")
print(f"Adversarial Accuracy:{100 * correct_adv / total:.2f}%")


100%|██████████| 100/100 [06:33<00:00,  3.94s/it]

Clean Accuracy:      82.63%
Adversarial Accuracy:26.06%
